In [1]:
# https://adventofcode.com/2018/day/22

using DataStructures
using Memoize


In [2]:
# part 1

const DEPTH = 3879
const TARGET = (8, 713) .+ 1
const GRID = TARGET .+ 100     # pad the grid for off-grid movement

const ROCKY = 0
const WET = 1
const NARROW = 2

@memoize function geoindex(x, y)
    if (x, y) == (1, 1) || (x, y) == TARGET
        return 0
    elseif y == 1
        return (x - 1) * 16807
    elseif x == 1
        return (y - 1) * 48271
    else
        return prod((geoindex.([x, x - 1], [y - 1, y]) .+ DEPTH) .% 20183)
    end
end

index = [geoindex(x, y) for x in 1:GRID[1], y in 1:GRID[2]]
erosion = (index .+ DEPTH) .% 20183
risk = erosion .% 3

sum(risk[1:TARGET[1], 1:TARGET[2]])


6323

In [3]:
# part 2

const GEAR = 0
const TORCH = 1
const NEITHER = 2

# find the valid neighbors of a point on the grid
function neighbors(u)
    x, y, z = u
    return [# first, find 1-unit manhattan distance points
            [(i, j, z)
             for i in x - 1:x + 1, j in y - 1:y + 1
             if i in 1:GRID[1] && j in 1:GRID[2]
                && (i, j) != (x, y)
                && (i == x || j == y)]; 
            # add neighbors for same point, different equipment
            [(x, y, k) 
             for k in [GEAR, TORCH, NEITHER] 
             if k != z]]
end

# compute the transition costs, for use in pathfinding
function cost(u, v)
    x, y, z = u
    i, j, k = v
    
    # same position, different equip, must be valid for the position
    if z != k
        if k == GEAR && risk[x, y] in [ROCKY, WET]
            return 7
        elseif k == TORCH && risk[x, y] in [ROCKY, NARROW]
            return 7
        elseif k == NEITHER && risk[x, y] in [WET, NARROW]
            return 7
        end
    # same equipment, different position, equipment must be valid
    elseif risk[i, j] == ROCKY && z in [GEAR, TORCH]
        return 1
    elseif risk[i, j] == WET && z in [GEAR, NEITHER]
        return 1
    elseif risk[i, j] == NARROW && z in [TORCH, NEITHER]
        return 1
    end
                                
    # incompatible position/equipment transition
    return Inf
end

# dijkstra's shortest path algorithm
queue = PriorityQueue()
distance = Dict()
previous = Dict()
                            
source = (1, 1, TORCH)
target = (TARGET[1], TARGET[2], TORCH)

for x in 1:GRID[1], y in 1:GRID[2], z in [GEAR, TORCH, NEITHER]
    v = (x, y, z)
    distance[v] = v == source ? 0 : Inf
    previous[v] = nothing
    enqueue!(queue, v, distance[v])
end

while !isempty(queue)
    u = dequeue!(queue)
    for v in neighbors(u)
        alt = distance[u] + cost(u, v)
        if alt < distance[v]
            distance[v] = alt
            previous[v] = u
            queue[v] = alt
        end
    end
end

distance[target]


982